In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

In [2]:
#Inputs
timeseries_file = 'INSIVUMEH_Meteo.csv'
ts_1970_raw = 'INSIVUMEH_Coban_1970_raw.csv'
# Constants
LRs = np.arange(1972, 2024, 4)
Months = np.arange(1, 13, 12)
#Output
file_ts70 = 'INSIVUMEH_Coban_1970_synth.csv'

In [3]:
#functions
def load_1970s_ts(ts_1970_raw):
    ts = np.loadtxt(ts_1970_raw, delimiter=',')
    ts_len = len(ts)
    Year   = ts[:, 0]
    Month  = ts[:, 1]
    Day    = ts[:, 2]
    Prcp   = ts[:, 3]
    T_avg  = ts[:, 4]
    T_max  = ts[:, 5]
    T_min  = ts[:, 6]
    return ts_len, Year, Month, Day, Prcp, T_avg, T_max, T_min


def load_timeseries_file(timeseries_file):
    INSIVUMEH_Coban = np.loadtxt(timeseries_file, delimiter=',')

    #INSIVUMEH_Coban = np.loadtxt('Meteo_only_temp_var.csv', delimiter=',')
    #Año	Mes	Día	LR	T_max	T_min	T_mean	Rh	CloudFrac13	Rad24	Rad6	Rad12	Rad18	
    # Wind_d	Wind_v	Press	ENSO	Precip	EvapPot
    ts_len = len(INSIVUMEH_Coban)
    Year  = INSIVUMEH_Coban[:, 0]
    Month = INSIVUMEH_Coban[:, 1]
    Day   = INSIVUMEH_Coban[:, 2]
    LR    = INSIVUMEH_Coban[:, 3]
    T_max = INSIVUMEH_Coban[:, 4]
    T_min = INSIVUMEH_Coban[:, 5]
    T_avg = INSIVUMEH_Coban[:, 6]
    RH    = INSIVUMEH_Coban[:, 7]
    Cld   = INSIVUMEH_Coban[:, 8] / 13 
    Rad24 = INSIVUMEH_Coban[:, 9]
    Rad6  = INSIVUMEH_Coban[:, 10]
    Rad12 = INSIVUMEH_Coban[:, 11]
    Rad18 = INSIVUMEH_Coban[:, 12]
    Wnd_d = INSIVUMEH_Coban[:, 13]
    Wnd_spd =INSIVUMEH_Coban[:, 14]
    Presh_in = INSIVUMEH_Coban[:, 15]
    Presh = (Presh_in - (113)) * 1.333223874 #Shifting and converting to mb
    #ENSO  = INSIVUMEH_Coban[:, 16]
    Prcp  = INSIVUMEH_Coban[:, 17]
    #Epot  = INSIVUMEH_Coban[:, 18]
    return ts_len, Year, Month, Day, LR, T_max, T_min, T_avg, RH, Cld, Rad24, Rad6, Rad12, Rad18, Wnd_d, Wnd_spd, Presh, Prcp


#Functions to fill timeseries70
def lin_func(x, y):
    mask = ~np.isnan(x) & ~np.isnan(y)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x[mask], y[mask])
    return slope, intercept, mask
    #return r_value

def predict_y(x, m, b, mask):
    y_lin = m * x[mask] + b
    return y_lin
    
def add_noise_y(x_in, y_in, y_lin_in, y_lin_out, mask):
    sigma = np.sqrt(np.sum((y_in[mask]-y_lin_in)**2) / len(y_in[mask]))
    rnds = np.random.normal(0, sigma, len(y_lin_out))
    y_noisy = y_lin_out + rnds 
    return y_noisy

def gen_synth_y(x_in, y_in, x_out):
    m, b, mask_in = lin_func(x_in, y_in)
    y_lin_in = predict_y(x_in, m, b, mask_in)
    mask_out = ~np.isnan(x_out)
    y_lin_out = predict_y(x_out, m, b, mask_out)
    y_noisy_out = add_noise_y(x_in, y_in, y_lin_in, y_lin_out, mask_in)
    y_noisy_out[y_noisy_out > max(y_in)] = max(y_in)
    y_noisy_out[y_noisy_out < min(y_in)] = min(y_in)
    return y_noisy_out


In [4]:
#run --> 
#Outs
ts_len70, Year70, Month70, Day70, Prcp70, T_avg70, T_max70, T_min70 = load_1970s_ts(ts_1970_raw)
#Ins
ts_len, Year, Month, Day, LR, T_max, T_min, T_avg, RH, Cld, Rad24, Rad6, Rad12, Rad18, Wnd_d, Wnd_spd, Presh, Prcp = load_timeseries_file(timeseries_file)

#LRs
LRs70 = np.in1d(Year70, LRs) * 1

#Fill with zeros
RHs70 = np.zeros(ts_len70)
Clds70 = np.zeros(ts_len70)
Presh70 = np.zeros(ts_len70)
Wnd_spd70 = np.zeros(ts_len70)
Wnd_d70 = np.zeros(ts_len70)
ENSOs70 = np.zeros(ts_len70)
Epot70 = np.zeros(ts_len70)
Rad24_70 = np.zeros(ts_len70)
Rad6_70  = np.zeros(ts_len70)
Rad12_70 = np.zeros(ts_len70)
Rad18_70 = np.zeros(ts_len70)

for i in range(1, 13):
    #TS Meteo variables 
    RHs70[Month70==i] = gen_synth_y(T_avg[Month==i], RH[Month==i], T_avg70[Month70==i])
    Clds70[Month70==i]  = gen_synth_y(T_max[Month==i], Cld[Month==i], T_max70[Month70==i])
    Presh70[Month70==i]  = gen_synth_y(T_max[Month==i], Presh[Month==i], T_max70[Month70==i])
    
    #Rads
    Rad24_70[Month70==i] = Rad24[Month==i][0]
    Rad6_70[Month70==i] = Rad6[Month==i][0]
    Rad12_70[Month70==i] = Rad6[Month==i][0]
    Rad18_70[Month70==i] = Rad6[Month==i][0]
    
    #Wind direction
    Wnd_d_month = Wnd_d[Month==i]
    mask = ~np.isnan(Wnd_d_month)
    Wnd_d70[Month70==i] = abs(np.ones(len(Wnd_d70[Month70==i])) * np.mean(Wnd_d_month[mask]) + (np.random.normal(0, np.std(Wnd_d_month[mask]), len(Wnd_d70[Month70==i]))))
    Wnd_d70[Wnd_d70>360] = 360

    #Wind speed 
    Wnd_spd_month = Wnd_spd[Month==i]
    mask = ~np.isnan(Wnd_spd_month)
    Wnd_spd70[Month70==i] = abs(np.ones(len(Wnd_d70[Month70==i])) * np.mean(Wnd_spd_month[mask]) + (np.random.normal(0, np.std(Wnd_spd_month[mask]), len(Wnd_d70[Month70==i]))))
    Wnd_spd70[Wnd_spd70 > max(Wnd_spd)] = max(Wnd_spd)
    Wnd_spd70[Wnd_spd70 < min(Wnd_spd)] = min(Wnd_spd)


timeseries70 = np.vstack((Year70, Month70, Day70, LRs70, T_max70, T_min70, T_avg70, RHs70, Clds70, Rad24_70, Rad6_70, Rad12_70, Rad18_70, Wnd_d70, Wnd_spd70, Presh70, ENSOs70, Prcp70, Epot70)).T
file_ts70 = 'INSIVUMEH_Coban_1970_synth.csv'
np.savetxt(file_ts70,timeseries70, delimiter=',')